In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import pickle
import napari
import numpy as np
import pandas as pd

from skimage.io import imread

import btrack
from btrack.constants import BayesianUpdates

sys.path.append('../libraries')
import input_functions as inp_f

In [2]:
info_file_path = r'Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\info_B02.txt'

In [3]:
# read the file
info_file = open(info_file_path, 'r')
info_lines = info_file.readlines()
info_file.close()

# read info about the data frame
exp_dir,df_name = inp_f.read_df_info(info_lines)

df_dir = os.path.join(exp_dir,'df')
save_dir = df_dir

frames_to_exclude = inp_f.read_frames_2_exclude(info_lines)
#frames_to_exclude = eval(frames_to_exclude)

modelPath = os.path.join(exp_dir,'code','libraries','cell_config.json')

## Read in the data frame objects data

In [4]:
data_df = pd.read_pickle(os.path.join(df_dir,df_name))

In [5]:
data_df.head()

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,bbox-3,image,mean_intensity-0_nuc,file,t,centroid-0_ring,centroid-1_ring,mean_intensity-0_ring,size_x,size_y
0,10,2486,26.693886,2479.874497,1.076862,70.784005,44.762487,1,2448,53,2514,"[[False, False, False, False, False, False, Fa...",1655.198311,WellB02_ChannelCFP_Seq0000 - Stitched_000_labe...,0,30.831257,2481.340883,642.251416,13435,13428
1,11,1001,13.434565,2828.053946,1.210713,44.682983,30.771292,1,2804,34,2848,"[[True, True, True, True, True, True, True, Tr...",697.327672,WellB02_ChannelCFP_Seq0000 - Stitched_000_labe...,0,20.406953,2828.672802,583.652352,13435,13428
2,12,2095,21.589499,5195.892601,1.431365,67.398337,39.774593,1,5162,42,5230,"[[False, False, False, False, False, False, Fa...",2267.723628,WellB02_ChannelCFP_Seq0000 - Stitched_000_labe...,0,25.437272,5195.978076,742.729598,13435,13428
3,13,2861,37.156239,9237.178259,-0.864351,72.801658,50.067969,7,9205,68,9269,"[[False, False, False, False, False, False, Fa...",1573.735757,WellB02_ChannelCFP_Seq0000 - Stitched_000_labe...,0,37.036723,9237.263653,721.075330,13435,13428
4,14,1048,34.223282,1490.215649,0.307027,48.521121,28.160505,8,1476,56,1507,"[[False, False, False, False, False, False, Fa...",933.001908,WellB02_ChannelCFP_Seq0000 - Stitched_000_labe...,0,33.216968,1490.027816,726.929068,13435,13428


In [6]:
len(data_df)

581734

In [7]:
# create a structure suitable for tracking

# choose objects 
sel_vector = [not(x in frames_to_exclude) for x in data_df.t]

objects_gen = data_df.loc[sel_vector,['label','area','centroid-1','centroid-0','major_axis_length','minor_axis_length','t']]

objects_gen.columns=['ID', 'area', 'x', 'y', 'major_axis_length','minor_axis_length','t']
objects_gen['z']=0
objects_gen['label']=5
objects_gen['prob']=0
objects_gen['dummy']=False
objects_gen['states']=0

objects_gen.head()

,ID,area,x,y,major_axis_length,minor_axis_length,t,z,label,prob,dummy,states
0,10,2486,2479.874497,26.693886,70.784005,44.762487,0,0,5,0,False,0
1,11,1001,2828.053946,13.434565,44.682983,30.771292,0,0,5,0,False,0
2,12,2095,5195.892601,21.589499,67.398337,39.774593,0,0,5,0,False,0
3,13,2861,9237.178259,37.156239,72.801658,50.067969,0,0,5,0,False,0
4,14,1048,1490.215649,34.223282,48.521121,28.160505,0,0,5,0,False,0


In [8]:
len(objects_gen)

581734

## Tracking proper

In [9]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(modelPath)
    
    # approximate
    tracker.update_method = BayesianUpdates.APPROXIMATE
    tracker.max_search_radius = 100

    # append the objects to be tracked
    tracker.append(objects_gen)

    # set the volume (Z axis volume is set very large for 2D data)
    tracker.volume=((0, data_df.size_x[0]), (0, data_df.size_y[0]), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    # optional: get the data in a format for napari
    data, properties, graph = tracker.to_napari(ndim=2)
    # pickle Napari data
    with open(os.path.join(df_dir,'track.pkl'),'wb') as f:
        pickle.dump([data,properties,graph],f)

[INFO][2023/08/14 02:52:16 PM] Loaded btrack: C:\Users\gases\.conda\envs\livecell_tracking\Lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2023/08/14 02:52:16 PM] Starting BayesianTracker session
[INFO][2023/08/14 02:52:16 PM] Loading configuration file: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\code\libraries\cell_config.json
[INFO][2023/08/14 02:52:16 PM] Objects are of type: <class 'pandas.core.frame.DataFrame'>
[WARNING][2023/08/14 02:52:23 PM] `track_interactive` will be deprecated. Use `track` instead.
[INFO][2023/08/14 02:52:23 PM] Starting tracking... 
[INFO][2023/08/14 02:52:23 PM] Update using: ['MOTION']
[INFO][2023/08/14 02:52:23 PM] Tracking objects in frames 0 to 99 (of 188)...
[INFO][2023/08/14 02:52:38 PM]  - Timing (Bayesian updates: 61.00ms, Linking: 46.00ms)
[INFO][2023/08/14 02:52:38 PM]  - Probabilities (Link: 0.99988, Lost: 1.00000)
[INFO][2023/08/14 02:52:38 PM]  - Stats (Active: 7385, Lost: 473630, Conflicts resolved: 28414)
[INFO][2023/08/14 02:52:38 PM] Tr

## Merging objects and tracking information

In [10]:
trackDataAll = pd.DataFrame(data,columns=['track_id','t','x','y'])
trackDataAll['parent'] = properties['parent']
trackDataAll['generation'] = properties['generation']
trackDataAll['root'] = properties['root']

In [11]:
len(trackDataAll)

628275

In [12]:
trackDataAll

,track_id,t,x,y,parent,generation,root
0,1.0,0.0,8371.940594,12394.912183,1,0,1
1,2.0,0.0,8310.065474,9081.284669,2,0,2
2,2.0,1.0,8318.763926,9068.812429,2,0,2
3,3.0,0.0,8308.896731,10454.902416,3,0,3
4,3.0,1.0,8321.149721,10440.769627,3,0,3
...,...,...,...,...,...,...,...
628270,169691.0,187.0,6555.289863,3068.840021,169691,0,169691
628271,169692.0,187.0,6615.023224,5469.521858,169692,0,169692
628272,169693.0,187.0,4216.636550,10883.990418,169693,0,169693
628273,169694.0,187.0,6665.465623,2667.466741,169694,0,169694


In [13]:
allData = pd.merge(left=data_df,right=trackDataAll,left_on=['centroid-0','centroid-1','t'],right_on=['x','y','t'],how='left')

In [14]:
print(f'Number of all objects: {len(allData)}')

Number of all objects: 581734


In [15]:
allData.columns

Index(['label', 'area', 'centroid-0', 'centroid-1', 'orientation',
       'major_axis_length', 'minor_axis_length', 'bbox-0', 'bbox-1', 'bbox-2',
       'bbox-3', 'image', 'mean_intensity-0_nuc', 'file', 't',
       'centroid-0_ring', 'centroid-1_ring', 'mean_intensity-0_ring', 'size_x',
       'size_y', 'track_id', 'x', 'y', 'parent', 'generation', 'root'],
      dtype='object')

In [16]:
# check how many objects doesn't have a track_id
test = np.sum(allData.track_id!=allData.track_id)
print(f'Number of objects without track_id: {test}')

Number of objects without track_id: 27


## Be careful!!!

In [17]:
# consider removing
allData = allData.loc[allData.track_id==allData.track_id,:]
print(f'Number of all objects: {len(allData)}')

Number of all objects: 581707


## Define promising tracks

This part is manual at the moment.

In [18]:
my_tracks = set(allData.track_id)
print(len(my_tracks))

169695


In [19]:
allData['accepted'] = False
allData['rejected'] = False
allData['promise'] = False

# mark tracks longer than 100 as promising
tracks_set = set(allData.track_id)

track_len=[]
promise_list = []
for track in tracks_set:
    
    # prepare signals for this track
    sel_signal = allData.loc[allData.track_id == track,['t','mean_intensity-0_nuc','mean_intensity-0_ring']]
    sel_signal.sort_values(by='t',inplace=True)
    sel_mean = sel_signal.rolling(9,min_periods = 9,center=True).mean()
    
    # test - length
    track_test = len(sel_signal)>50
    
    track_len.append(len(sel_signal))
    
    # test - DHB presence
    dhb_test = np.sum(sel_mean['mean_intensity-0_nuc'] > (sel_mean['mean_intensity-0_ring']+100)) > 10
    
    if (track_test and dhb_test):
        
        promise_list.append(track)
        
        allData.loc[allData.track_id==track,'promise'] = True

In [20]:
len(promise_list)

722

## Create columns for requested annotations

In [21]:
# get info about the tags (for annotating points on the tracks)
flag_list = inp_f.read_flags(info_lines,df=allData)

for flag in flag_list:
    
    allData[flag['flag_column']]=False

In [22]:
# save df
allData.to_pickle(os.path.join(df_dir,df_name))
allData.to_csv(os.path.join(df_dir,df_name.replace('pkl','csv')),index=False)